In [4]:
"""Naval Fate.

Usage:
  naval_fate.py ship new <name>...
  naval_fate.py ship <name> move <x> <y> [--speed=<kn>]
  naval_fate.py ship shoot <x> <y>
  naval_fate.py mine (set|remove) <x> <y> [--moored | --drifting]
  naval_fate.py (-h | --help)
  naval_fate.py --version

Options:
  -h --help     Show this screen.
  --version     Show version.
  --speed=<kn>  Speed in knots [default: 10].
  --moored      Moored (anchored) mine.
  --drifting    Drifting mine.

"""
from docopt import docopt
def main():
    args = docopt(__doc__)
    print(args)

In [107]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import pytorch_lightning as pl
from torchvision.ops import MLP

class RndDataset(Dataset):

    def __init__(self, size, num_samples):

        self.len = num_samples
        self.data = torch.randn(num_samples, size)
        self.label = torch.randint(2,(num_samples,)).long()

    def __getitem__(self, index):
        return self.data[index], self.label[index]

    def __len__(self):
        return self.len

class BoringModel1(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.network1 = MLP(4, [3])
        self.network2 = MLP(4, [3])
        self.criterion = nn.CrossEntropyLoss(reduction='mean')

    def training_step(self, batch, batch_idx):
        
        inputs, labels = batch
                
        logits1 = self.network1(inputs)
        logits2 = self.network2(inputs)
        
        sup1 = self.criterion(logits1, labels)
        sup2 = self.criterion(logits2, labels)
        
        with torch.no_grad():
            _, m2 = torch.max(logits2, dim=1)
        unsup2=self.criterion(logits1, m2)
        
        with torch.no_grad():
            _, m1 = torch.max(logits1, dim=1)
        unsup1=self.criterion(logits2, m1)
        
        loss = unsup1+unsup2+sup1+sup2
        return {"loss": loss}

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=0.1)
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1)
        return [optimizer], [lr_scheduler]
    
class BoringModel2(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.network1 = MLP(4, [3])
        self.network2 = MLP(4, [3])
        self.criterion = nn.CrossEntropyLoss(reduction='mean')
        self.automatic_optimization = False

    def training_step(self, batch, batch_idx):
        
        opt1, opt2 = self.optimizers()
        opt1.zero_grad()
        opt2.zero_grad()
                        
        inputs, labels = batch
                
        logits1 = self.network1(inputs)
        logits2 = self.network2(inputs)
        
        sup1 = self.criterion(logits1, labels)
        sup2 = self.criterion(logits2, labels)
        
        _, m2 = torch.max(logits2, dim=1)
        unsup2=self.criterion(logits1, m2)
        _, m1 = torch.max(logits1, dim=1)
        unsup1=self.criterion(logits2, m1)
        loss = unsup1+unsup2+sup1+sup2
        
        self.manual_backward(loss)
        opt1.step()
        opt2.step()
        
        return {"loss": loss}

    def configure_optimizers(self):
        opt1 = torch.optim.SGD(self.network1.parameters(), lr=0.1)
        opt2 = torch.optim.SGD(self.network2.parameters(), lr=0.1)
        lr_sch1 = torch.optim.lr_scheduler.StepLR(opt1, step_size=1)
        lr_sch2 = torch.optim.lr_scheduler.StepLR(opt2, step_size=1)
        return [opt1, opt2], [lr_sch1, lr_sch2]

In [110]:
torch.manual_seed(1)

model1 = BoringModel1()

#print('\n Model 1 Network1 weights :', model1.network1.weight)
#print('\n Model 1 Network1 bias :', model1.network1.bias)

trainer1 = pl.Trainer(
    accelerator='cpu',
    devices=1,
    max_epochs=1,
)

trainset1 = RndDataset(4, 2)
train1 = DataLoader(trainset1, shuffle=False, batch_size=1)

trainer1.fit(model1, train1)

print('\n Model 1 Network1 weights :', model1.network1[0].weight)
print('\n Model 1 Network1 bias :', model1.network2[0].weight)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type             | Params
-----------------------------------------------
0 | network1  | MLP              | 15    
1 | network2  | MLP              | 15    
2 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
30        Trainable params
0         Non-trainable params
30        Total params
0.000     Total estimated model params size (MB)


Epoch 0: 100%|██████████| 2/2 [00:00<00:00, 186.32it/s, v_num=75]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 2/2 [00:00<00:00, 66.77it/s, v_num=75] 

 Model 1 Network1 weights : Parameter containing:
tensor([[ 0.4152,  0.0570, -0.2193,  0.3584],
        [-0.5608,  0.0951,  0.0291,  0.1991],
        [ 0.0020, -0.1342,  0.1291, -0.0438]], requires_grad=True)

 Model 1 Network1 bias : Parameter containing:
tensor([[ 0.0792,  0.2773, -0.0794,  0.5394],
        [ 0.0884, -0.3792, -0.1495, -0.1117],
        [-0.2727, -0.1721, -0.0511,  0.2237]], requires_grad=True)


In [111]:
torch.manual_seed(1)

model2 = BoringModel2()

trainer2 = pl.Trainer(
    accelerator='cpu',
    devices=1,
    max_epochs=1,
)

trainset2 = RndDataset(4, 2)
train2 = DataLoader(trainset2, shuffle=False, batch_size=1)

trainer2.fit(model2, train2)

print('\n Model 2 Network1 weights :', model2.network1[0].weight)
print('\n Model 2 Network1 bias :', model2.network2[0].weight)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type             | Params
-----------------------------------------------
0 | network1  | MLP              | 15    
1 | network2  | MLP              | 15    
2 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
30        Trainable params
0         Non-trainable params
30        Total params
0.000     Total estimated model params size (MB)


Epoch 0: 100%|██████████| 2/2 [00:00<00:00, 186.52it/s, v_num=76]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 2/2 [00:00<00:00, 100.15it/s, v_num=76]

 Model 2 Network1 weights : Parameter containing:
tensor([[ 0.4152,  0.0570, -0.2193,  0.3584],
        [-0.5608,  0.0951,  0.0291,  0.1991],
        [ 0.0020, -0.1342,  0.1291, -0.0438]], requires_grad=True)

 Model 2 Network1 bias : Parameter containing:
tensor([[ 0.0792,  0.2773, -0.0794,  0.5394],
        [ 0.0884, -0.3792, -0.1495, -0.1117],
        [-0.2727, -0.1721, -0.0511,  0.2237]], requires_grad=True)


In [9]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)

100%|██████████| 26421880/26421880 [00:04<00:00, 5623426.33it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 55838918.61it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




100%|██████████| 4422102/4422102 [00:00<00:00, 5329747.95it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 21876673.75it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



NameError: name 'dataloader' is not defined

In [2]:
import rasterio
import numpy as np
file_name = '/work/OT/ai4geo/DATA/DATASETS/DIGITANIE/Biarritz/Biarritz_EPSG32630_1.tif'
file_name = '/data/DIGITANIE/Biarritz/Biarritz_EPSG32630_1.tif'
with rasterio.open(file_name) as src:
    print(src.meta)
    band1 = src.read(1)
    print('Band1 has shape', band1.shape)
    height = band1.shape[0]
    width = band1.shape[1]
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rasterio.transform.xy(src.transform, rows, cols)
    lons= np.array(xs)
    lats = np.array(ys)
    print('lons shape', lons.shape)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 2048, 'height': 2048, 'count': 4, 'crs': CRS.from_epsg(32630), 'transform': Affine(0.4996690458118865, 0.0, 620734.67813581,
       0.0, -0.4996690458118865, 4816930.568377791)}
Band1 has shape (2048, 2048)
lons shape (2048, 2048)


In [3]:
lons = lons[:100, :100]
lats = lats[:100, :100]

In [4]:
import pyproj as proj
import utm

# setup your projections
crs_src = proj.Proj(init='epsg:32630') # assuming you're using WGS84 geographic
crs_dst = proj.Proj(init='epsg:4326 ') # use a locally appropriate projected CRS

# then cast your geographic coordinate pair to the projected system
input_lon, input_lat = proj.transform(crs_src, crs_dst, lons, lats)
input_lon, input_lat

/d/pfournie/dl_toolbox/venv/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/d/pfournie/dl_toolbox/venv/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_3854905/320663268.py:9: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  input_lon, inpu

(array([[-1.50669398, -1.5066878 , -1.50668162, ..., -1.5060947 ,
         -1.50608852, -1.50608235],
        [-1.50669409, -1.50668791, -1.50668173, ..., -1.50609481,
         -1.50608864, -1.50608246],
        [-1.5066942 , -1.50668802, -1.50668184, ..., -1.50609492,
         -1.50608875, -1.50608257],
        ...,
        [-1.50670473, -1.50669855, -1.50669237, ..., -1.50610546,
         -1.50609928, -1.5060931 ],
        [-1.50670484, -1.50669866, -1.50669248, ..., -1.50610557,
         -1.50609939, -1.50609321],
        [-1.50670495, -1.50669877, -1.50669259, ..., -1.50610568,
         -1.5060995 , -1.50609333]]),
 array([[43.49555068, 43.4955506 , 43.49555052, ..., 43.49554285,
         43.49554277, 43.49554269],
        [43.49554618, 43.4955461 , 43.49554602, ..., 43.49553835,
         43.49553827, 43.49553819],
        [43.49554168, 43.4955416 , 43.49554152, ..., 43.49553385,
         43.49553377, 43.49553369],
        ...,
        [43.49511441, 43.49511433, 43.49511425, ..., 4

In [5]:
utm.from_latlon(input_lat, input_lon)

(array([[620734.92797094, 620735.42763999, 620735.92730903, ...,
         620783.39586839, 620783.89553743, 620784.39520648],
        [620734.92797094, 620735.42763999, 620735.92730903, ...,
         620783.39586839, 620783.89553743, 620784.39520648],
        [620734.92797094, 620735.42763999, 620735.92730903, ...,
         620783.39586839, 620783.89553743, 620784.39520648],
        ...,
        [620734.92797094, 620735.42763999, 620735.92730903, ...,
         620783.39586839, 620783.89553743, 620784.39520648],
        [620734.92797094, 620735.42763999, 620735.92730903, ...,
         620783.39586839, 620783.89553743, 620784.39520648],
        [620734.92797094, 620735.42763999, 620735.92730903, ...,
         620783.39586839, 620783.89553743, 620784.39520648]]),
 array([[4816930.31872322, 4816930.31872322, 4816930.31872322, ...,
         4816930.31872322, 4816930.31872322, 4816930.31872322],
        [4816929.81905417, 4816929.81905417, 4816929.81905417, ...,
         4816929.81905417, 48

In [ ]:
np.multiply(np.cos(lats), np.cos(lons))
np.multiply(np.cos(lats), np.sin(lons))

In [1]:
import rasterio

def read_window_from_big_raster(window, path, raster_path):
    with rasterio.open(path) as image_file:
        with rasterio.open(raster_path) as raster_file:
            left, bottom, right, top = rasterio.windows.bounds(
                window, 
                transform=image_file.transform
            )
            rw = rasterio.windows.from_bounds(
                left, bottom, right, top, 
                transform=raster_file.transform
            )
            image = raster_file.read(
                window=rw, 
                out_dtype=np.float32
            )
    return image

In [43]:
from rasterio.windows import Window
import numpy as np
window = Window(0, 0, 10000, 10000)
path = p+f'/Biarritz_EPSG32630_9.tif'
big_path = p+'/BIARRITZ_20140902_T_TOA_reproj-EPSG:32630.tif'
img = read_window_from_big_raster(window, path, big_path)

In [44]:
def read_window_basic(window, path):
    with rasterio.open(path) as image_file:
        image = image_file.read(window=window, out_dtype=np.float32)
    return image

In [50]:
window = Window(0, 0, 10000, 10000)
img2 = read_window_basic(window,big_path)

In [51]:
img2.shape

(4, 10000, 10000)

In [1]:
import rasterio
p = '/work/OT/ai4usr/fournip/digitanie/Toulouse/toulouse_full_tiled.tif'
ds = rasterio.open(p)
print(ds)

<open DatasetReader name='/work/OT/ai4usr/fournip/digitanie/Toulouse/toulouse_full_tiled.tif' mode='r'>


In [2]:
a = ds.statistics(bidx=1)

In [3]:
a

Statistics(min=-0.0029133798088878393, max=1.5431257486343384, mean=0.09988372604258908, std=0.05581474629124038)

In [4]:
a.min

-0.0029133798088878393